In [ ]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Loading data which will be submitted 


In [ ]:
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

# Load Train and Test Data

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv("../input/digit-recognizer/test.csv")
print(test.shape)
test.head()

In [ ]:
y_train = train["label"]
x_train = train.drop(labels = ["label"],axis = 1)

In [ ]:
plt.figure(figsize=(15,7))
sns.countplot(y_train, palette = "icefire")
plt.title("Number of Digit Classes")
y_train.value_counts()

In [ ]:
random_img = x_train.iloc[27].to_numpy()
random_img = random_img.reshape((28,28))
plt.imshow(random_img,cmap='gray')
plt.title(train.iloc[27,0])
plt.axis("off")
plt.show()

# Normalization
* We should perform normalization images to make process, in our case CNN model,faster
* With normalization, gray scale images are obtained. 

In [ ]:
x_train = x_train / 255.0
test = test / 255.0
print("x_train shape : ", x_train.shape)
print("test : shape : ", test.shape)

# Reshaping
*  Train and test images (28 x 28)
* We reshape all data to 28x28x1 3D matrices.
* Keras needs an extra dimension in the end which correspond to channels. Our images are gray scaled so it use only one channel.

In [ ]:
x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print("x_train shape : ", x_train.shape)
print("test : shape : ", test.shape)

# Label Encoding

In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes = 10)

# Spliting Test and Train Data
* In my model, I prefer to split test and train data with 1/20 ratio.
* Train size is 95%.
* Test size is 5%.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val ,y_train, y_val = train_test_split(x_train, y_train, test_size = 0.05, random_state = 2)
print("x_train shape : ", x_train.shape)
print("x_test : shape : ", x_val.shape)
print("y_train shape : ", y_train.shape)
print("y_test : shape : ", y_val.shape)

# Implementing Our Model


**Needed Libraries**

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

**Initializing The Model**

In [ ]:
model = Sequential()

**Buiding The Model Structure**

In [ ]:

model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = "Same",activation = "relu",
                input_shape =(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = "Same",activation = "relu",))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "Same",activation = "relu",))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "Same",activation = "relu",))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 128, kernel_size = (5,5), padding = "Same",activation = "relu",))
model.add(Conv2D(filters = 128, kernel_size = (5,5), padding = "Same",activation = "relu",))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
print (model.summary())

**Optimizer**

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

**Compiling The Model**

In [ ]:
model.compile(optimizer = optimizer , 
              loss = "categorical_crossentropy", metrics=["accuracy"])

**Hyperparatemers**

In [ ]:
epochs = 30  # for better result increase the epochs
batch_size = 32

**Data Augmentation**

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=0.2,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.2, # Randomly zoom image 5%
        width_shift_range=0.2,  # randomly shift images horizontally 5%
        height_shift_range=0.2,  # randomly shift images vertically 5%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

**Processing Data with The Model (Fitting)**

In [ ]:
datagen.fit(x_train)

In [ ]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_val,y_val), 
                              steps_per_epoch=x_train.shape[0] // batch_size)

# Visualization of Result

**Test Loss**

In [ ]:
# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='b', label="Validation Accuracy")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()
plt.figure()

**Test Accuracy**

In [ ]:
plt.plot(history.history['val_accuracy'], color='b', label="Valdation Accuracy")
plt.title("Test Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
plt.figure()

**Confusion Matrix**

In [ ]:
# confusion matrix
import seaborn as sns
# Predict the values from the validation dataset
Y_pred = model.predict(x_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
pred = np.argmax(Y_pred,axis = 1) 

# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Write The Results to Output File

In [ ]:
predictions = model.predict(test)
predictions = np.argmax(predictions, axis=1)
sample_submission['Label'] = predictions
sample_submission.to_csv("output3.csv", index=False)